<a target="_blank" href="https://colab.research.google.com/github/Tensor-Reloaded/Neural-Networks-Template-2024/blob/main/Lab01/Assignment1.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# **Assignment 1 (10 points)**

## **Solving a linear system in python**

In this homework, you will familiarize yourself with key linear algebra con-
cepts and Python programming by solving a system of linear equations. You
will explore multiple methods for solving such systems, including Cramer’s rule
and matrix inversion. By the end of this assignment, you will have a good un-
derstanding of how to represent and manipulate matrices and vectors in Python.

We begin with the following system of 3 linear equations with 3 unknowns:
$$ 2x + 3y - z = 5 $$
$$ x - y + 4z = 6 $$
$$ 3x + y + 2z = 7 $$

This system can be vectorized in the following form:
$$ A \cdot X = B $$
where:
$$
A = \begin{bmatrix}
2 & 3 & -1 \\
1 & -1 & 4 \\
3 & 1 & 2
\end{bmatrix}, \quad 
X = \begin{bmatrix}
x \\
y \\
z
\end{bmatrix}, \quad 
B = \begin{bmatrix}
5 \\
6 \\
7
\end{bmatrix}
$$

**Considerations**
- do not use any linear algebra framework such as $numpy$
- use python lists as data structures for matrices and vectors
- experiment with other values for the coefficients and free terms

### **1. Parsing the System of Equations (1 point)**

The first task is to implement a Python script that reads a system of linear equations from a text file and parses it into a matrix $A$ and a vector $B$. You will use the input format described below to extract the coefficients for $A$ and $B$.

**Input File Format**
```text
2x + 3y - z = 5
x - y + 4z = 6
3x + y + 2z = 7
```

Note that the coefficients are always in the order x, y and z and the terms are always space separated

In [4]:
import pathlib
import re

def load_system(path: pathlib.Path) -> tuple[list[list[float]], list[float]]:
    A = []
    B = []
    
    equation_pattern = re.compile(r"([+-]?\d*)x\s*([+-]\s*\d*)y\s*([+-]\s*\d*)z\s*=\s*([+-]?\d+)")
    
    with path.open() as file:
        for line in file:
            line = line.strip()
            match = equation_pattern.match(line)
            
            if match:
                x_coeff = match.group(1).replace(" ", "").strip()
                y_coeff = match.group(2).replace(" ", "").strip()
                z_coeff = match.group(3).replace(" ", "").strip()
                constant = match.group(4).strip()
                
                x_coeff = int(x_coeff) if x_coeff not in ["", "+", "-"] else int(x_coeff + "1")
                y_coeff = int(y_coeff) if y_coeff not in ["", "+", "-"] else int(y_coeff + "1")
                z_coeff = int(z_coeff) if z_coeff not in ["", "+", "-"] else int(z_coeff + "1")
                constant = int(constant)
                
                A.append([x_coeff, y_coeff, z_coeff])
                B.append(constant)
    
    return A, B

A, B = load_system(pathlib.Path("system.txt"))
print(f"{A=} {B=}")


A=[[2, 3, -1], [1, -1, 4], [3, 1, 2]] B=[5, 6, 7]


### **2. Matrix and Vector Operations (5 points)**

Once you have successfully parsed the matrix and vector, complete the following exercises to manipulate and understand basic matrix and vector operations. Write Python functions for each of these tasks:

#### 2.1. Determinant

Write a function to compute the determinant of matrix $A$. Recall one of the formulae for the determinant of a $3x3$ matrix:
$$ \text{det}(A) = a_{11}(a_{22}a_{33} - a_{23}a_{32}) - a_{12}(a_{21}a_{33} - a_{23}a_{31}) + a_{13}(a_{21}a_{32} - a_{22}a_{31}) $$

In [50]:
def determinant(matrix: list[list[float]]) -> float:
  if len(matrix) == 2:
        return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]
  det = 0
  for c in range(len(matrix)):
    minor = [row[:c] + row[c+1:] for row in matrix[1:]]
    det += ((-1) ** c) * matrix[0][c] * determinant(minor)

  return det  
print(f"{determinant(A)=}")


determinant(A)=14


#### 2.2. Trace

Compute the sum of the elements along the main diagonal of matrix $A$. For a matrix $A$, this is:
$$ \text{Trace}(A) = a_{11} + a_{22} + a_{33} $$

In [6]:
def trace(matrix: list[list[float]]) -> float:
   trace=0
   for i in range(len(matrix)):
    trace+=matrix[i][i]
   return trace 
print(f"{trace(A)=}")

trace(A)=3


#### 2.3. Vector norm

Compute the Euclidean norm of vector $B$, which is:
$$ ||B|| = \sqrt{b_1^2 + b_2^2 + b_3^2} $$

In [52]:
import math

def norm(vector: list[float]) -> float:
    norm=0
    for b in vector:
     norm +=b*b
    return math.sqrt(norm)
print(f"{norm(B)=}")

norm(B)=10.488088481701515


#### 2.4. Transpose of matrix

Write a function to compute the transpose of matrix $A$. The transpose of a matrix $A$ is obtained by swapping its rows and columns.
    

In [31]:
def transpose(matrix: list[list[float]]) -> list[list[float]]:
    transposed = [[matrix[j][i] for j in range(len(matrix))] for i in range(len(matrix[0]))]
    return transposed
print(f"{transpose(A)=}")

transpose(A)=[[2, 1, 3], [3, -1, 1], [-1, 4, 2]]


#### 2.5. Matrix-vector multiplication

Write a function that multiplies matrix $A$ with vector $B$.

In [53]:
def multiply(matrix: list[list[float]], vector: list[float]) -> list[float]:
 multiply_matrix=[]
 for row in matrix:
    row_sum = sum(row[i] * vector[i] for i in range(len(vector)))
    multiply_matrix.append(row_sum)


 return multiply_matrix
print(f"{multiply(A, B)=}")

multiply(A, B)=[21, 27, 35]


### **3. Solving using Cramer's Rule (1 point)**

Now that you have explored basic matrix operations, solve the system of linear equations using Cramer's rule.

**Cramer's Rule:**

Cramer's rule allows you to solve for each unknown $x$, $y$, and $z$ using determinants. For example:
$$ x = \frac{\text{det}(A_x)}{\text{det}(A)}, \quad y = \frac{\text{det}(A_y)}{\text{det}(A)}, \quad z = \frac{\text{det}(A_z)}{\text{det}(A)} $$
where $A_x$, $A_y$, and $A_z$ are matrices formed by replacing the respective column of matrix $A$ with vector $B$.

In [54]:
import copy

def replace_col(matrix: list[list[float]], column:int, new_column:list[float]):
  copy_matrix = copy.deepcopy(matrix)
  for i in range(len(matrix)):
    copy_matrix[i][column] = new_column[i]
  return copy_matrix


def solve_cramer(matrix: list[list[float]], vector: list[float]) -> list[float]:
  det = determinant(matrix)
  solutions = []
 
  for i in range(len(matrix)):
    det_i = determinant(replace_col(matrix, i, vector))  
    solutions.append(det_i / det)
  
  return solutions
print(f"{solve_cramer(A, B)=}")

solve_cramer(A, B)=[0.35714285714285715, 2.0714285714285716, 1.9285714285714286]


### **4. Solving using Inversion (3 points)**

Finally, solve the system by computing the inverse of matrix $A$ and multiplying it by vector $B$.
$$ A \cdot X = B \rightarrow X = A^{-1} \cdot B $$
**Adjugate Method for Matrix Inversion:**

To find the inverse of matrix $ A $, you can use the adjugate method:
$$ A^{-1} = \frac{1}{\text{det}(A)} \times \text{adj}(A) $$
where $\text{adj}(A)$ is the adjugate (or adjoint) matrix, which is the transpose of the cofactor matrix of $ A $.

**Cofactor Matrix:**

The cofactor matrix is a matrix where each element is replaced by its cofactor. The cofactor of an element $a_{ij}$ is given by:
$$ (-1)^{i+j} \times \text{det}(M_{ij}) $$
where $M_{ij}$ is the minor of element $a_{ij}$, which is the matrix obtained by removing the $i$-th row and $j$-th column from matrix $A$.

In [57]:
import copy

def minor(matrix: list[list[float]], i: int, j: int) -> list[list[float]]:
 return [row[:j]+row[j+1:] for row in (matrix[:i]+matrix[i+1:])]
         

def cofactor(matrix: list[list[float]]) -> list[list[float]]:
 cofactor=[]
 for i in range(len(matrix)):
    row = []
    for j in range(len(matrix)):
       minor_det=determinant(minor(matrix,i,j))
       sign=(-1)**(i+j)
       row.append(sign*minor_det)

    cofactor.append(row)
 return cofactor
def adjoint(matrix: list[list[float]]) -> list[list[float]]:
 return transpose(cofactor(matrix))

def inverse(matrix: list[list[float]]) -> list[list[float]]:
    return [[element / determinant(matrix) for element in row] for row in adjoint(matrix)]

def solve(matrix: list[list[float]], vector: list[float]) -> list[float]:
        inv_matrix = inverse(matrix)
        solution=[]
        solution= [sum(row[i] * vector[i] for i in range(len(vector))) for row in inv_matrix]
        return solution
print(cofactor(A))
print(adjoint(A))
print(f"{solve(A, B)=}")


[[1, 4], [3, 2]]
[[-6, 10, 4], [-7, 7, 7], [11, -9, -5]]
[[-6, -7, 11], [10, 7, -9], [4, 7, -5]]
solve(A, B)=[0.3571428571428572, 2.0714285714285716, 1.9285714285714284]
